In [1]:
import numpy as np
from numpy import linalg as LA
import pandas as pd
from math import radians, cos, sin, asin, sqrt, isnan
import random
from ortools.sat.python import cp_model
import time

In [2]:
#-----------------------------------EXTRACT THE DATA-----------------------------------
#start_time = time.time()

# users-melbcbd-generated.csv contains:
# •  Latitude-Longitude
# of the users in the Melbourne CBD area.
requests_path = '..\\eua-dataset\\users\\'
U = pd.read_csv(requests_path + 'users-test.csv') # test with 8 users
#R = pd.read_csv(requests_path + 'users-test-more-users.csv') # test with 14 users
#R = pd.read_csv(requests_path + 'users-melbcbd-generated.csv') # test with 816 users

# site-optus-melbCBD.csv contains:
# •  SiteID-Latitude-Longitude-Name-State-LicensingAreaID-PostCode-SitePrecision-Elevation-HCISL2
# of all Optus BS in Melbourne CBD area (edge-servers)
nodes_path = '..\\eua-dataset\\edge-servers\\'
#N = pd.read_csv(nodes_path + 'serverstest.csv') # test with 3 servers
N_src = pd.read_csv(nodes_path + 'serverstest.csv') # test with 3 servers
N_dest = pd.read_csv(nodes_path + 'serverstest.csv') # test with 3 servers
#N = pd.read_csv(nodes_path + 'serverstest-more-servers.csv') # test with 6 servers
#N = pd.read_csv(nodes_path + 'site-optus-melbCBD.csv') # test with 125 servers

In [3]:
#-----------------------------------INPUTS-----------------------------------

#Incoming requests to node i
#Function(rows) x Node(column) [request/sec]
lambda_ri=[[1,0,0],[1,0,1],[1,1,1],[2,0,0]] 

#Amount of request received in time-slot
R=np.sum(lambda_ri)
coordinates_R = pd.read_csv(requests_path + 'users-test.csv')

#Identifies which user sent the request [U x R]
req_u =np.zeros([len(U),R])
req_u=[[1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,1]]


#Ammount of arrivals per node
ammount_arrivals_n = []
for i in range(len(N_src)):
    ammount_arrivals_n.append(np.sum(np.transpose(lambda_ri)[i]))

 
# Memory of function: m_f
m1 = 1
m2 = 2
m3 = 3
m4 = 4
m_f = [m1,m2,m3,m4]

#show which requests are assigned to each function [F x R]
req_dist = np.zeros([len(m_f),R])

r = 0
while r<R:
    for i in range(len(N_src)):
        for f in range(len(m_f)):
            dif = 0
            if (lambda_ri[f][i]*m_f[f])/m_f[f]==1:
                req_dist[f][r]=1
                r=r+1
            else:
                dif = lambda_ri[f][i] 
                while dif >0:
                    req_dist[f][r]=1
                    r=r+1
                    dif = dif-1

#matrix that assignes a function memory to each request [F x N]
m_request= np.empty((len(m_f),R))
for f in range(len(lambda_ri)):
  for r in range (R):
        m_request[f][r]=m_f[f]*req_dist[f][r]

# # Sort the requests by their memory requirement --- returns position of the [] where request is found
# m_index=[]

# for r in range (R):
#   for f in range (len(m_f)):
#     if m_request[f][r]!=0:
#       m_index.append(m_request[f][r])
       
# final_index =sorted(range(len(m_index)), key=lambda a: m_index[a]) 
  
#Maximum allowed network delay for function f:  phi_f
phi_f=[]

In [4]:
#-----------------------------------INFRASTRUCTURED DATA-----------------------------------

#Memory available in node j: M_j
M_j = [10,10,10]

#Cores on node j: U_j
U_j=[16,16,16]

In [5]:
#-----------------------------------MONITORED DATA-----------------------------------
#Network delay between nodes i and j
delta_ij = []

#Cores used by node j per single request: u_rj 
#u_rj = [[2,2,2,2,2,2,2,2],[2,2,2,2,2,2,2,2],[2,2,2,2,2,2,2,2]]
u_rj = [2,2,2,2,2,2,2,2]

In [6]:
#-----------------------------------VARIABLES-----------------------------------

dij = [] # Set of requests conected to node n
#U_ni_d = [] # useless?

#1 if request r arrives to node i [N x R]
loc_arrival_r=np.zeros([len(N_src),R])
r=0
while r<R:
    for i in range (len(N_src)):
        for n in range (ammount_arrivals_n[i]):
            loc_arrival_r[i][r]=1
            r=r+1

In [7]:
#-----------------------------------HAVERSINE-----------------------------------

def haversine(lon1, lat1, lon2, lat2):
  
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [8]:
#-----------------------------------COVERAGE REQUEST-NODE-----------------------------------

#radius = np.round(np.random.uniform(0.1,0.15,len(S)),3) # in km
radius = np.full(len(N_src), 0.03)

for i in range(len(N_src)):
  node_latitude = N_src.iloc[i]['LATITUDE']
  node_longitude = N_src.iloc[i]['LONGITUDE']
  temp = []
  #t_distance =[] # useless??
  for r in range(R):
    for u in range(len(U)):

      if req_u[u][r]==1:
        request_latitude = U.iloc[u]['Latitude']
        request_longitude = U.iloc[u]['Longitude']

        dist_geo = haversine(node_longitude, node_latitude, request_longitude, request_latitude)
        #t_distance.append(np.round(dist_geo,3)) # useless??
        temp.append(np.round(dist_geo,3))
        # if dist_geo <= radius[i]:
        #   temp.append(1)
        
        # else:
        #   temp.append(0)
       
  dij.append(temp)
  #U_ni_d.append(t_distance) # useless??


In [9]:
#---------------------------------DISTANCE BETWEEN NODES---------------------------------

#radius = np.round(np.random.uniform(0.1,0.15,len(S)),3) # in km
radius = np.full(len(N_src), 0.03)

for i in range(len(N_src)):
  node1_latitude = N_src.iloc[i]['LATITUDE']
  node1_longitude = N_src.iloc[i]['LONGITUDE']
  temp_dist = []
  for j in range(len(N_dest)):
    node2_latitude = N_dest.iloc[j]['LATITUDE']
    node2_longitude = N_dest.iloc[j]['LONGITUDE']

    dist_geo_nodes = haversine(node1_longitude, node1_latitude, node2_longitude,node2_latitude)
    #print(dist_geo)
    #print(radius[i]+radius[j])
    temp_dist.append(np.round(dist_geo_nodes,3))
       
  delta_ij.append(temp_dist)

for r1 in range(len(N_src)):
  temp_delay = []
  for r2 in range(len(N_dest)):
    if r1==r2:
      temp_delay.append(0)
    else:
      temp_delay.append(radius[r1]+radius[r2])
  phi_f.append(temp_delay)
    

In [10]:
#---------------------------------- CP MODEL ----------------------------------
model = cp_model.CpModel()

#-----------------------------------SOLVER VARIABLES-----------------------------------

# x_j,r = True if request r is allocated to node j
x = {}
for j in range(len(N_dest)):
    for r in range(R): ## 
        x[j, r] = model.NewBoolVar(f'c[{j}][{r}]')

# c_f,j = True if container f (fa,fb,fc) is deployed on node j
c = {}
for f in range(len(lambda_ri)):
    for j in range(len(N_dest)):
        c[f, j] = model.NewBoolVar(f'c[{f}][{j}]')

# y_j = True if node j is used
# y_j = False otherwise
y = {}
for j in range(len(N_dest)): 
    y[j] = model.NewBoolVar(f'c[{j}]')
 
print(model.ModelStats())

satisfaction model '':
#Variables: 39
  - 39 Booleans in [0,1]



In [11]:
#-----------------------------------CONSTRAINTS-----------------------------------

#Controls if request r can be managed by node j
for r in range(R):
    for j in range(len(N_dest)):
        #if dij[j][r]==0:
        if dij[j][r]>= radius[j]:
            model.Add(x[j, r]==0)
        
        #if loc_arrival_r[j][r]==1 and U_ni[j][r]==1:
         #    model.Add(x[j, r]==1)
  
#Proximity constraint (node i-node j) 
for i in range(len(N_src)):
    for r in range(R):
        for j in range(len(N_dest)):
                if delta_ij[i][j]> phi_f[i][j]:
                    model.Add(
                    x[j, r]==0
                    )  

#Container deployment: True if container of function f is deployed on node j
for f in range(len(lambda_ri)):
    for j in range(len(N_dest)):
        model.Add(sum([x[j,r] for r in range(R)])<= c[f,j]*1000)

# Capacity constraint (memory)
for j in range(len(N_dest)):
    model.Add(
                sum([
                     m_f[f]* c[f,j] for f in range(len(lambda_ri))
                    ]) <= M_j[j]*y[j]
                )
     
# Avoid resource contention
for j in range(len(N_dest)):
        model.Add(
                sum([sum([x[j,r]*lambda_ri[f][i]*u_rj[r] for r in range(R)]) for i in range(len(N_src))
                ]) <= U_j[j]*y[j]   
            )  
 
# Contraint family (each request can be allocated just once)
for r in range(R):
    model.Add(sum([x[j, r] for j in range(len(N_dest))]) <= 1)

 
print(model.ModelStats())

satisfaction model '':
#Variables: 39
  - 39 Booleans in [0,1]
#kLinear1: 11
#kLinear3: 8
#kLinearN: 18 (#terms: 150)


In [12]:
#---------------------------------- CP SOLVER ----------------------------------
solver = cp_model.CpSolver()

#---------------------------------- OBJECTIVE FUNCTIONS ---------------------------------

# Maximize the number of allocated requests
objective_max = []

for j in range(len(N_dest)):
    for r in range(R):
        objective_max.append(x[j, r])
model.Maximize(sum(objective_max))

solver.Solve(model)
max_requests = solver.ObjectiveValue()

# Hint (speed up solving)
for j in range(len(N_dest)):
     for r in range(R):
        model.AddHint(x[j,r], solver.Value(x[j,r]))
    
# Constraint previous objective
model.Add(
    sum([
        x[j, r] for j in range(len(N_dest)) for r in range(R)
    ]) == round(solver.ObjectiveValue())
) 

# Minimize the number of nodes used
objective_min = []

for j in range(len(N_dest)):
    objective_min.append(y[j])
model.Minimize(sum(objective_min))

print(model.ModelStats())

optimization model '':
#Variables: 39 (#bools:3 in objective)
  - 39 Booleans in [0,1]
#kLinear1: 11
#kLinear3: 8
#kLinearN: 19 (#terms: 174)


In [13]:
#-----------------------------------CALL THE SOLVER-----------------------------------
status = solver.Solve(model)
x_rj = np.zeros([len(N_dest),R])
#-----------------------------------DISPLAY THE SOLUTION-----------------------------------

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('SOLUTION:')
    print(f'Objective value: {max_requests} requests have been allocated to {solver.ObjectiveValue()} nodes\n')
    for r in range(R):
        for j in range(len(N_dest)):
            if int(solver.Value(x[j,r])) == 1:
                print(f'x[{j},{r}]: Request {r} has been allocated to node {j}')
                x_rj[j][r]=1
                
    print('----------------------------------------------------------------------')
    for f in range(len(lambda_ri)):
        for j in range(len(N_dest)):
            if int(solver.Value(c[f,j])) == 1:
                print(f'c[{f},{j}]: Function {f} has been deployed on node {j}')

    print('----------------------------------------------------------------------')
    for j in range(len(N_dest)):
        if int(solver.Value(y[j])) == 1:
                print(f'y[{j}]: Node {j} is used') 
    #---------------------------------------------CERO VARIABLES----------------------------------------------------
    print('  ')
    print('----------------------------------------------------------------------')
    print('VARIABLES IN ZERO')
    print('----------------------------------------------------------------------')
    for r in range(R):
        for j in range(len(N_dest)):
            if int(solver.Value(x[j,r])) == 0:
                print(f'x[{j},{r}]')
    print('----------------------------------------------------------------------')
    for f in range(len(lambda_ri)):
        for j in range(len(N_dest)):
            if int(solver.Value(c[f,j])) == 0:
                print(f'c[{f},{j}]')

    print('----------------------------------------------------------------------')
    for j in range(len(N_dest)):
        if int(solver.Value(y[j])) == 0:
                print(f'y[{j}]')
else:
    print('The problem does not have an optimal solution.')

#print("\n--- Run time: %s seconds ---" % round((time.time() - start_time),2))

SOLUTION:
Objective value: 8.0 requests have been allocated to 2.0 nodes

x[2,0]: Request 0 has been allocated to node 2
x[1,1]: Request 1 has been allocated to node 1
x[1,2]: Request 2 has been allocated to node 1
x[1,3]: Request 3 has been allocated to node 1
x[1,4]: Request 4 has been allocated to node 1
x[2,5]: Request 5 has been allocated to node 2
x[2,6]: Request 6 has been allocated to node 2
x[2,7]: Request 7 has been allocated to node 2
----------------------------------------------------------------------
c[0,1]: Function 0 has been deployed on node 1
c[0,2]: Function 0 has been deployed on node 2
c[1,1]: Function 1 has been deployed on node 1
c[1,2]: Function 1 has been deployed on node 2
c[2,1]: Function 2 has been deployed on node 1
c[2,2]: Function 2 has been deployed on node 2
c[3,1]: Function 3 has been deployed on node 1
c[3,2]: Function 3 has been deployed on node 2
----------------------------------------------------------------------
y[1]: Node 1 is used
y[2]: Node 

In [14]:
#-----------------------------------Normalization-----------------------------------
mat_mul=np.dot(req_dist,np.transpose(x_rj))
x_fj=np.zeros([len(lambda_ri),len(N_dest)])
for j in range(len(N_dest)):
    for f in range(len(lambda_ri)):
        x_fj[f][j]=mat_mul[f][j]/sum(mat_mul[f])